In [1]:
import glob

import numpy as np
import pandas as pd

from grafting_classifier import GraftingClassifier
from sklearn.linear_model import SGDClassifier
from ogfs_classifier import OGFSClassifier
from osfs_classifier import OSFSClassifier
from dpp_classifier import DPPClassifier
from dpp_classifier_mitra import DPPClassifier as DPPClassifier2
from dpp_classifier_ogfs import DPPClassifier as DPPClassifier3

from sklearn.metrics import log_loss, accuracy_score

#import dask.dataframe as dd
#import dask.array as da

In [2]:
class_train = glob.glob("NIPS/*_train.csv")
print(class_train)

['NIPS\\arcene_train.csv', 'NIPS\\dexter_train.csv', 'NIPS\\dorothea_train.csv', 'NIPS\\gisette_train.csv', 'NIPS\\madelon_train.csv']


In [3]:
def train_label(fname):
    targetname = fname.replace(".csv", ".labels")
    return pd.read_csv(targetname)

In [4]:
def get_performance(mod, fpath, base=False):
    train1 = pd.read_csv(fpath).fillna(0)
    col_sel = [x for x in train1.columns.tolist() if x != 'target']
    train1 = train1[col_sel]
    y = np.array(train_label(fpath)).flatten()
    
    # simulate streaming...
    # try splitting into groups of ~10,
    # if there is no splits, try ~5.
    train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/10.0) + 1)
    if len(train1_cols) == 1:
        train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/5.0) + 1)
    all_cols = []

    #mod = GraftingClassifier(max_iter=5)
    if base:
        mod.fit(train1, y)
        results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
        return results
    
    # lets normalise the dataset...
    train1 = (train1 - train1.mean())/(np.maximum(train1.std(), 1))
    for idx, collist in enumerate(train1_cols):
        if idx == 0:
            column_list = list(np.array(list(train1.columns))[collist])
            mod.fit(train1[column_list], y)
            all_cols.extend(list(collist))
        else:
            all_cols.extend(list(collist))
            column_list = list(np.array(list(train1.columns))[all_cols])
            mod.partial_fit(train1[column_list], y)
    
    results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
    return results

In [5]:
def create_models():
    return [
    ('Grafting', GraftingClassifier(max_iter=5, random_state=42)), 
    ('DPP', DPPClassifier(max_iter=5, random_state=42)), 
    #('DPP2', DPPClassifier2(max_iter=5, random_state=42)),
    ('DPP3', DPPClassifier3(max_iter=5, random_state=42)),
    ('OGFS', OGFSClassifier(max_iter=5, random_state=42)),
    ('OSFS', OSFSClassifier(max_iter=5, random_state=42)),
    ('Base', SGDClassifier(loss='log', max_iter=5, random_state=42))
]

In [6]:
ex_dat = class_train[3]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

NIPS\gisette_train.csv (6000, 5000)
Grafting {'accuracy': 0.98899999999999999, 'logloss': 0.036959451043201558, 'feat_dim': (138,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\kernel_approximation.py:470: UserWarning: n_components > n_samples. This is not possible.
n_components was set to n_samples, which results in inefficient evaluation of the full kernel.
  warnings.warn("n_components > n_samples. This is not possible.\n"
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_in

DPP {'accuracy': 0.76266666666666671, 'logloss': 0.53303751471298244, 'feat_dim': (6,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:79: RuntimeWarning: divide by zero encountered in true_divide
  X = s_b/s_w
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:98: RuntimeWarning: divide by zero encountered in double_scalars
  prev_score = np.sum(curr_u1)/np.sum(curr_u2)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:93: RuntimeWarning: divide by zero encountered in double_scalars
  score = ((np.sum(test_u1)/np.sum(test_u2)) - prev_score)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:93: RuntimeWarning: invalid value encountered in double_scalars
  score = ((np.sum(test_u1)/np.sum(test_u2)) - prev_score)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:143: RuntimeWarning: divide by zero encountered in true_divide
  eval2 = s_b/s_w
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.p

DPP3 {'accuracy': 0.76183333333333336, 'logloss': 0.49872233855613485, 'feat_dim': (32,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\SPEC.py:150: RuntimeWarning: divide by zero encountered in double_scalars
  w_fea[i] = np.sum(a1 * s[0:n_samples-1])/(1-np.power(np.dot(np.transpose(F_hat), v), 2))
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\ogfs_classifier.py:80: RuntimeWarning: invalid value encountered in double_scalars
  score = ((np.sum(test_u1)/np.sum(test_u2)) - prev_score)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\ogfs_classifier.py:119: RuntimeWarning: invalid value encountered in double_scalars
  t_stat = (mu - X[idx])/(sigma/np.sqrt(U))
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\ogfs_classifier.py:66: RuntimeWarning: invalid value encountered in true_divide
  X = s_b/s_w
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\ogfs_classifier.py:85: RuntimeWarning: invalid value encountered in double_scalars
  prev_score = np.sum(curr_u1)/np.sum(curr_u2)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\str

OGFS {'accuracy': 0.99116666666666664, 'logloss': 0.028828522153572341, 'feat_dim': (166,)}
OSFS {'accuracy': 0.91916666666666669, 'logloss': 0.21737861485611581, 'feat_dim': (22,)}
Base {'accuracy': 0.9986666666666667, 'logloss': 0.046051701859881909, 'feat_dim': (5000,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
